In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import ElectraTokenizer, TFElectraForSequenceClassification

In [7]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")

In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=2)

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_electra_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, 
          validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 248s 6s/step - loss: 0.6233 - accuracy: 0.7106 - val_loss: 0.4087 - val_accuracy: 0.8899
Epoch 2/10
39/39 [==============================] - 231s 6s/step - loss: 0.3216 - accuracy: 0.9002 - val_loss: 0.2508 - val_accuracy: 0.9188
Epoch 3/10
39/39 [==============================] - 232s 6s/step - loss: 0.1883 - accuracy: 0.9398 - val_loss: 0.2312 - val_accuracy: 0.9242
Epoch 4/10
39/39 [==============================] - 220s 6s/step - loss: 0.1044 - accuracy: 0.9721 - val_loss: 0.2426 - val_accuracy: 0.9224
Epoch 5/10
39/39 [==============================] - 219s 6s/step - loss: 0.0830 - accuracy: 0.9739 - val_loss: 0.2825 - val_accuracy: 0.9224
Epoch 6/10
39/39 [==============================] - 220s 6s/step - loss: 0.0517 - accuracy: 0.9884 - val_loss: 0.2720 - val_accuracy: 0.9242
Epoch 6: early stopping


In [15]:
model.load_weights(checkpoint_filepath)

In [16]:
model.evaluate(dict(X_test_tokenized), y_test)

44/44 [==============================] - 19s 436ms/step - loss: 0.2229 - accuracy: 0.9249


[0.22291795909404755, 0.9248554706573486]

In [17]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 21s 439ms/step
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       620
           1       0.95      0.91      0.93       764

    accuracy                           0.92      1384
   macro avg       0.92      0.93      0.92      1384
weighted avg       0.93      0.92      0.92      1384

